In [4]:
import requests
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [28]:
raw_data = pd.read_csv('Profeco_Catalog.csv')
raw_data.head(1)

,Unnamed: 0,_id,cadenaComercial,catalogo,categoria,direccion,estado,fechaRegistro,giro,latitud,longitud,marca,municipio,nombreComercial,precio,presentacion,producto,razonSocial,rfc
0,0,571e4c002f232201006c6cac,HIPERMERCADO SORIANA,BASICOS,TORTILLAS Y DERIVADOS DEL MAIZ,"BLVD. ADOLFO LOPEZ MATEOS 1100, ESQ. CALAFIA, ...",BAJA CALIFORNIA,2015-02-10,TIENDA DE AUTOSERVICIO,32.641719,-115.469582,S/M,MEXICALI,SORIANA HIPER SUCURSAL CALAFIA,11.9,1 KG. GRANEL,TORTILLA DE MAIZ,NaN,NaN


In [29]:
master_frame = raw_data.drop(columns = ['giro','razonSocial','rfc','catalogo','Unnamed: 0','direccion','nombreComercial']).reset_index(drop=True).rename(index=str, columns={"_id": "ID", "cadenaComercial": "Cadena", "categoria": "Categoría", "estado": "Estado", "fechaRegistro": "Fecha", "latitud": "Latitud", "longitud": "Longitud", "marca": "Marca", "municipio": "Municipio", "precio": "Precio", "presentacion": "Presentación", "producto": "Producto"})
master_frame.head(1)

,ID,Cadena,Categoría,Estado,Fecha,Latitud,Longitud,Marca,Municipio,Precio,Presentación,Producto
0,571e4c002f232201006c6cac,HIPERMERCADO SORIANA,TORTILLAS Y DERIVADOS DEL MAIZ,BAJA CALIFORNIA,2015-02-10,32.641719,-115.469582,S/M,MEXICALI,11.9,1 KG. GRANEL,TORTILLA DE MAIZ


In [31]:
year_registry = master_frame.groupby('Fecha')['Fecha'].count()
year_registry

Fecha
2012-01-10      9063
2013-01-10     40539
2015-02-10    131192
2016-05-10     38788
Name: Fecha, dtype: int64

In [12]:
category = master_frame.groupby('Categoría')['Categoría'].count().nlargest(10)
category

Categoría
MEDICAMENTOS                         41061
ARTS. PARA EL CUIDADO PERSONAL       16291
APARATOS ELECTRICOS                  13713
DETERGENTES Y PRODUCTOS SIMILARES    12235
DERIVADOS DE LECHE                   11074
CARNES FRIAS SECAS Y EMBUTIDOS       10901
APARATOS ELECTRONICOS                10070
CONDIMENTOS                           9655
REFRESCOS ENVASADOS                   9650
FRUTAS Y LEGUMBRES PROCESADAS         9319
Name: Categoría, dtype: int64

In [119]:
#Estados = Master["Estado"].nunique()
Estados = Master.drop_duplicates("Estado")
#Estados.head(100)
Estados = Estados["Estado"].tolist()
Estados

['DURANGO',
 'GUERRERO',
 'TAMAULIPAS',
 'MICHOACÁN DE OCAMPO',
 'AGUASCALIENTES',
 'BAJA CALIFORNIA',
 'BAJA CALIFORNIA SUR',
 'CHIAPAS',
 'CHIHUAHUA',
 'CAMPECHE',
 'SONORA',
 'QUINTANA ROO',
 'MORELOS',
 'COAHUILA DE ZARAGOZA']

In [16]:
category_state = master_frame[master_frame['Categoría'] == 'MEDICAMENTOS'].groupby(\
                            ['Estado', 'Categoría'])['Categoría'].count()
category_state

Estado                           Categoría   
BAJA CALIFORNIA                  MEDICAMENTOS     639
BAJA CALIFORNIA SUR              MEDICAMENTOS     200
CAMPECHE                         MEDICAMENTOS     454
CHIAPAS                          MEDICAMENTOS     289
CHIHUAHUA                        MEDICAMENTOS     361
COAHUILA DE ZARAGOZA             MEDICAMENTOS      35
COLIMA                           MEDICAMENTOS    1258
DISTRITO FEDERAL                 MEDICAMENTOS    8088
DURANGO                          MEDICAMENTOS     344
GUANAJUATO                       MEDICAMENTOS    3154
GUERRERO                         MEDICAMENTOS     336
HIDALGO                          MEDICAMENTOS     951
JALISCO                          MEDICAMENTOS    2861
MICHOACÁN DE OCAMPO              MEDICAMENTOS     889
MÉXICO                           MEDICAMENTOS    6591
M��XICO                          MEDICAMENTOS       2
NUEVO LEÓN                       MEDICAMENTOS    2547
OAXACA                           MED

In [18]:
master_frame.groupby(master_frame['fechaRegistro'].date.dt.year)

KeyError: 'fechaRegistro'

# Master Data Frame

Table Ready to make analysis on any product, category, state, etc.

In [2]:
Master = Master.infer_objects()
#Master['Precio'] = Master.to_numeric(Master['Precio'], downcast='float')
#Master['Latitud'] = Master.to_numeric(Master['Latitud'], downcast='float')
#Master['Longitud'] = Master.to_numeric(Master['Longitud'], downcast='float')
Master.dropna()
Master.head(100)
#Master.dtypes


NameError: name 'Master' is not defined

ID               object
Cadena           object
Categoría        object
Estado           object
Fecha            object
Latitud         float64
Longitud        float64
Marca            object
Municipio        object
Precio          float64
Presentación     object
Producto         object
dtype: object

## -- Analysis
### --
-----